In [1]:
import numpy as np

In [2]:
from keras.datasets import mnist
from keras.utils import np_utils

def load_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_test  = X_test.reshape(X_test.shape[0], 28, 28, 1)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255
    X_test /= 255

    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    
    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_mnist()
print(X_train.shape)

Using TensorFlow backend.


(60000, 28, 28, 1)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# From https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [ ]:
model = Sequential()
# CNN layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))
model.add(Flatten())
# FC layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(.50))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.output_shape)

In [ ]:
model.fit(X_train, y_train,
        epochs=1,
        batch_size=32,
        verbose=1,
        validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print(score)

# TODO TODO TODO https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

# Structured model

In [10]:
def build_model():
    
    cnn_layers = [
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(.25),
        Flatten()
    ]

    fc_layers = [
        Dense(128, activation='relu'),
        Dropout(.50),
        Dense(10, activation='softmax')
    ]

    model = Sequential()
    for layer in cnn_layers: model.add(layer)
    for layer in fc_layers: model.add(layer)
    
    return model, cnn_layers, fc_layers

# Without transfer learning

In [7]:
model, cnn_layers, fc_layers = build_model()

for layer in cnn_layers: 
    layer.trainable = True
    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train,
        epochs=1,
        batch_size=32,
        verbose=1,
        validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print(score)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 9984/10000 [============================>.] - ETA: 0s[0.048684340486256403, 0.9839]


# Transfer learning

In [12]:
model, cnn_layers, fc_layers = build_model()

for layer in cnn_layers: 
    layer.trainable = False
    
for layer in fc_layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train,
        epochs=1,
        batch_size=32,
        verbose=1,
        validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print(score)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 9984/10000 [============================>.] - ETA: 0s[2.3087842872619628, 0.1056]
